# Amanogawa Notebook 02: Dark Morphology (Thin Demo)

This notebook demonstrates dark-lane morphology using the package API only.
No notebook-local scientific core function is used.

- Purpose: quick web/Colab trial
- Source of truth: `src/amanogawa/dark_morphology.py`
- Outputs: `outputs/notebooks/02_dark_morphology/`


In [ ]:
from __future__ import annotations

import json
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

ROOT_CANDIDATES = [Path("..").resolve(), Path(".").resolve()]
ROOT = next((p for p in ROOT_CANDIDATES if (p / "src" / "amanogawa").exists()), ROOT_CANDIDATES[0])
SRC_DIR = ROOT / "src"
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

from amanogawa import __version__
from amanogawa.dark_morphology import detect_dark_lane_mask_from_file
from amanogawa.detection import detect_stars_log
from amanogawa.io import load_image_gray

IMAGE_PATH = ROOT / "data" / "raw" / "IMG_5991.jpg"
OUT_DIR = ROOT / "outputs" / "notebooks" / "02_dark_morphology"
RESULT_DIR = OUT_DIR / "results"
FIG_DIR = OUT_DIR / "figures"
for p in [RESULT_DIR, FIG_DIR]:
    p.mkdir(parents=True, exist_ok=True)

print(f"Amanogawa version: {__version__}")
print(f"Image: {IMAGE_PATH}")
print(f"Output root: {OUT_DIR}")


In [ ]:
image, (width_px, height_px) = load_image_gray(IMAGE_PATH)
star_df = detect_stars_log(image, threshold=0.05, max_sigma=6.0, num_sigma=12)

dark_res = detect_dark_lane_mask_from_file(IMAGE_PATH, out_dir=RESULT_DIR)
mask_png_path = RESULT_DIR / "dark_lane_mask.png"
Image.fromarray((dark_res.combined_mask.astype(np.uint8) * 255)).save(mask_png_path)

summary = {
    "status": "ok",
    "image": IMAGE_PATH.name,
    "image_path": str(IMAGE_PATH),
    "width_px": int(width_px),
    "height_px": int(height_px),
    "num_stars_detected": int(len(star_df)),
    "dark_metrics": dark_res.metrics,
}
(RESULT_DIR / "dark_morphology_summary.json").write_text(json.dumps(summary, indent=2), encoding="utf-8")

print(f"Detected stars: {len(star_df)}")
print(f"Dark area fraction: {dark_res.metrics['dark_area_fraction']:.4f}")
print(f"Dark components: {dark_res.metrics['num_dark_components']}")


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
axes[0].imshow(image, cmap="gray")
if len(star_df) > 0:
    axes[0].scatter(star_df["x"], star_df["y"], s=5, c="red", alpha=0.5)
axes[0].set_title(f"Input + detected stars (N={len(star_df)})")
axes[0].set_axis_off()

axes[1].imshow(dark_res.combined_mask, cmap="gray")
axes[1].set_title("Dark-lane mask")
axes[1].set_axis_off()

plt.tight_layout()
plot_path = FIG_DIR / "dark_morphology_overview.png"
plt.savefig(plot_path, dpi=140)
plt.show()
print(f"Saved figure: {plot_path}")


In [ ]:
generated_files = sorted(str(p.relative_to(ROOT)) for p in OUT_DIR.rglob("*") if p.is_file())
print("Generated files:")
for p in generated_files:
    print(f" - {p}")
